In [1]:
!pip install -q langchain langchain-community langchain-openai faiss-cpu pypdf networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00


In [2]:
import os
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.graphs import NetworkxEntityGraph
from langchain_community.graphs.networkx_graph import KnowledgeTriple
from langchain.chains import GraphQAChain
from langchain.callbacks.tracers import ConsoleCallbackHandler
from google.colab import userdata

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(
    model="gpt-5-mini",
    api_key=OPENAI_API_KEY,
)

In [5]:
print("---  RAG 체인 구성 ---")
rag_documents = [
    Document(page_content="ACA의 동적 계획 수립 모듈은 '리플렉션(Reflection)' 메커니즘을 통해 실패 경험으로부터 학습하고, 다음 행동 계획을 최적화합니다.", metadata={"source": "ACA_research.pdf"})
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(rag_documents)
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY),
)
retriever = vectorstore.as_retriever()
rag_prompt = ChatPromptTemplate.from_template("문맥 정보를 바탕으로 다음 질문에 답변하세요:\n\n문맥: {context}\n\n질문: {question}")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": (lambda x: x["question"]) | retriever | format_docs,
        "question": (lambda x: x["question"]),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)
print("✓ RAG 체인 구성 완료")

---  RAG 체인 구성 ---
✓ RAG 체인 구성 완료


In [6]:
print("--- KG 체인 구성 ---")
graph = NetworkxEntityGraph()
graph.add_triple(KnowledgeTriple("Tom Hanks", "ACTED_IN", "Forrest Gump"))
graph.add_triple(KnowledgeTriple("Tom Hanks", "ACTED_IN", "Saving Private Ryan"))
graph.add_triple(KnowledgeTriple("Tom Hanks", "ACTED_IN", "Cast Away"))
kg_chain_internal = GraphQAChain.from_llm(llm=llm, graph=graph, verbose=False)

kg_chain = (lambda x: {"query": x["question"]}) | kg_chain_internal
print("✓ KG 체인 구성 완료")

--- KG 체인 구성 ---
✓ KG 체인 구성 완료


In [7]:
print("--- 일반 대화 체인 구성 중 ---")
general_prompt = ChatPromptTemplate.from_template("다음 질문에 친절하게 답변해줘: {question}")
general_chain = general_prompt | llm | StrOutputParser()
print("✓ 일반 대화 체인 구성 완료")

--- 일반 대화 체인 구성 중 ---
✓ 일반 대화 체인 구성 완료


In [8]:
print("--- 라우터 체인 구성 ---")
router_prompt_template = """사용자의 질문을 분석하여 어떤 전문가에게 보내야 할지 결정해주세요.
당신은 다음 세 가지 선택지 중 하나만 골라야 합니다: 'RAG', 'KG', 'GENERAL'

'RAG': 자율 인지 에이전트(ACA), AI 아키텍처, 기술 문서에 대한 질문일 경우.
'KG': 영화, 감독, 배우 간의 관계에 대한 질문일 경우.
'GENERAL': 위의 두 경우에 해당하지 않는 일반적인 대화나 인사일 경우.

사용자 질문: {question}
선택:"""
router_prompt = ChatPromptTemplate.from_template(router_prompt_template)
router_chain = router_prompt | llm | StrOutputParser()
print("✓ 라우터 체인 구성 완료")

--- 라우터 체인 구성 ---
✓ 라우터 체인 구성 완료


In [9]:
# --- RunnableBranch를 사용한 전체 체인 결합 ---
print("--- 전체 체인 결합 중 ---")
full_router_chain = RunnableBranch(
    (lambda x: "RAG" in x["topic"], rag_chain),
    (lambda x: "KG" in x["topic"], kg_chain),
    general_chain,
)
chain = {"topic": router_chain, "question": lambda x: x["question"]} | full_router_chain
print("✓ 전체 체인 결합 완료")

--- 전체 체인 결합 중 ---
✓ 전체 체인 결합 완료


In [10]:
print("--- 라우터 체인 실행 및 테스트 ---")
# 테스트 1: RAG 경로
print("## 테스트 1: RAG 경로 ##")
rag_question = "ACA의 동적 계획 수립은 어떻게 작동하나요?"
rag_answer = chain.invoke({"question": rag_question}, config={'callbacks': [ConsoleCallbackHandler()]})
print(f"질문: {rag_question}\n답변: {rag_answer}")

# 테스트 2: KG 경로
print("\n## 테스트 2: KG 경로 ##")
kg_question = "톰 행크스가 출연한 영화는?"
kg_answer = chain.invoke({"question": kg_question}, config={'callbacks': [ConsoleCallbackHandler()]})
print(f"질문: {kg_question}\n답변: {kg_answer.get('result') if isinstance(kg_answer, dict) else kg_answer}")

# 테스트 3: GENERAL 경로
print("\n## 테스트 3: GENERAL 경로 ##")
general_question = "오늘 날씨 정말 좋네요."
general_answer = chain.invoke({"question": general_question}, config={'callbacks': [ConsoleCallbackHandler()]})
print(f"질문: {general_question}\n답변: {general_answer}")

--- 라우터 체인 실행 및 테스트 ---
## 테스트 1: RAG 경로 ##
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "ACA의 동적 계획 수립은 어떻게 작동하나요?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<topic,question>] Entering Chain run with input:
{
  "question": "ACA의 동적 계획 수립은 어떻게 작동하나요?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<topic,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "ACA의 동적 계획 수립은 어떻게 작동하나요?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<topic,question> > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "ACA의 동적 계획 수립은 어떻게 작동하나요?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<topic,question> > chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > chain:RunnableParallel<topic,question> > chain:RunnableSequence > llm:ChatOp